### Práctica de Web Scraping

Se va a considerar una página web de wikipedia donde se encuentra un recopilatorio de las pinturas realizadas por Francisco Goya: https://es.wikipedia.org/wiki/Anexo:Cuadros_de_Goya

En esta página aparece una tabla en la que aparece cada pintura junto a información asociada a la pintura.

In [ ]:
from IPython.display import Image
Image(filename='Captura.png')

La tabla está formada por 6 columnas: Cuadro, Título, Fecha, Dimensiones, Museo y País.
Observa cómo es la estructura de una fila

Los enlaces a las imágenes se encuentran en el atributo src del elemento <img>. En el ejemplo anterior, el valor de src es:
//upload.wikimedia.org/wikipedia/commons/thumb/f/f1/El_ap%C3%B3stol_Santiago_y_sus_disc%C3%ADpulos_adorando_a_la_Virgen_del_Pilar.jpg/150px-El_ap%C3%B3stol_Santiago_y_sus_disc%C3%ADpulos_adorando_a_la_Virgen_del_Pilar.jpg

Para acceder a la imagen se debe añadir https: delante del valor recuperado:

https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/El_ap%C3%B3stol_Santiago_y_sus_disc%C3%ADpulos_adorando_a_la_Virgen_del_Pilar.jpg/150px-El_ap%C3%B3stol_Santiago_y_sus_disc%C3%ADpulos_adorando_a_la_Virgen_del_Pilar.jpg

Para recuperar el contenido de la página html de wikipedia, se puede hacer de la siguiente manera:

In [ ]:
import requests
url="https://es.wikipedia.org/wiki/Anexo:Cuadros_de_Goya"
r = requests.get(url)
html = r.text
print(html)

Se pide hacer un buscador que permita recuperar información de la página. Para ello se le mostrará al usuario dos opciones entre las que tiene que elegir: Buscar por país o Buscar por museo.

__Buscar por país[5 puntos] __
 
Si elige esta opción se le mostrará un listado de todos los paises en los que existe una pintura de Goya. Para facilitar la selección se asignará un número a cada páis, de manera que el usuario elija por número. Los países se pueden recuperar de la última columna de la tabla de la página de wikipedia. Una vez elegido el país, el programa mostrará una lista de todas las pinturas que cumplen ese requisito. De cada pintura se mostrará la información que aparece en las columnas: Título, Fecha, Dimensiones, y Museo. Así mismo, se creará una carpeta en el sistema de archivos, y se guardarán todas las imagenes de las pinturas que cumplen la condición de búsqueda sobre país.

__Buscar por museo[5 puntos] __
 
Si elige esta opción se le mostrará un listado de todos los museos en los que existe una pintura de Goya. Para facilitar la selección se asignará un número a cada museo, de manera que el usuario elija por número. Los museos se pueden recuperar de la penúltima columna de la tabla de la página de wikipedia. Una vez elegido el museo, el programa mostrará una lista de todas las pinturas que cumplen ese requisito. De cada pintura se mostrará la información que aparece en las columnas: Título, Fecha, Dimensiones, y País. Así mismo, se creará una carpeta en el sistema de archivos, y se guardarán todas las imagenes de las pinturas que cumplen la condición de búsqueda sobre museo.

__Nota: Para procesar la página html es obligatorio utilizar la libreria BeautifulSoup. No se pueden usar otras librerías no vistas en clase__

# Normas de entrega

* Fecha tope de entrega: 28/10/2018
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre WebScraping_GrupoX donde X será el número de grupo correspondiente.


In [ ]:
import requests

# Devuelve el contenido del documento html de la página web
def obtener_html():
    url = "https://es.wikipedia.org/wiki/Anexo:Cuadros_de_Goya"
    r = requests.get(url)
    html = r.text
    return html

In [ ]:
from bs4 import BeautifulSoup

# Escanea el documento html y devuelve todos los datos de la tabla en un diccionario con 6 pares clave-valor:
# 1 (url_imagenes: lista con las url de todas las imagenes)
# 2 (titulos: lista con todos los titulos de pinturas)
# 3 (fechas: lista con todas las fechas)
# 4 (dimensiones: lista con todas las dimensiones)
# 5 (museos: lista con todos los museos)
# 6 (paises: lista con todos los paises)
def obtener_datos():
    # Recuperar todas las etiquetas de inicio de fila de tabla
    html = obtener_html()
    soup = BeautifulSoup(html, 'html.parser')
    encabezado = soup.find("tr")
    filas = encabezado.find_all_next("tr")
    
    # Obtener los datos de la tabla del documento html
    lista_url_imagenes = []
    lista_titulos = []
    lista_fechas = []
    lista_dimensiones = []
    lista_museos = []
    lista_paises = []
    for fila in filas:
        celda_imagen = fila.find_next("td")
        celda_titulo = celda_imagen.find_next("td")
        celda_fecha = celda_titulo.find_next("td")
        celda_dimensiones = celda_fecha.find_next("td")
        celda_museo = celda_dimensiones.find_next("td")
        celda_pais = celda_museo.find_next("td")
        
        imagen_url = "https:" + celda_imagen.find_next('a', attrs = {'class':'image'}).find_next("img").get('src')
        museo = celda_museo.text.strip().replace("\n", "").capitalize()
        pais = celda_pais.find_next("a").text.strip().replace("\n", "")
        
        lista_url_imagenes.append(imagen_url)
        lista_titulos.append(celda_titulo.text)
        lista_fechas.append(celda_fecha.text)
        lista_dimensiones.append(celda_dimensiones.text)
        lista_museos.append(museo)
        lista_paises.append(pais)

    # Devolver todos los datos reunidos en un diccionario
    tabla = {}
    tabla["url_imagenes"] = lista_url_imagenes
    tabla["titulos"] = lista_titulos
    tabla["fechas"] = lista_fechas
    tabla["dimensiones"] = lista_dimensiones
    tabla["museos"] = lista_museos
    tabla["paises"] = lista_paises
    return tabla

In [ ]:
import os

# Prepara la carpeta de imágenes de pinturas cuyo nombre se pasa como parámetro
# En caso de que ya exista, borra todas las imágenes que haya en su interior
# En caso contrario, crea la carpeta
def preparar_carpeta_imagenes(nombre_carpeta):
    if os.path.exists(nombre_carpeta):
        for archivo in os.listdir(nombre_carpeta):
            os.remove(nombre_carpeta + "/" + archivo)
    else:
        os.mkdir(nombre_carpeta)

In [ ]:
# Muestra de forma indexada cada uno de los diferentes países contenidos en la tabla
# Pide al usuario que escoja uno y devuelve su nombre
def elegir_pais(tabla):
    # Mostrar de forma indexada cada uno de los diferentes países de la tabla
    listaPaisesDiferentes = []
    i = 1
    print("Lista de países:\n")
    for pais in tabla["paises"]:
        if pais not in listaPaisesDiferentes and pais != "":
            listaPaisesDiferentes.append(pais)
            print(str(i) + "\t: " + pais)
            i += 1
    # Devolver el país escogido por el usuario
    print("\nIntroduce un número de país")
    return listaPaisesDiferentes[int(input()) - 1]

In [ ]:
import urllib.request

# Muestra la informacion de todas las pinturas que se encuentren en el país que escoja el usuario
# Guarda una imagen de cada pintura en la carpeta "Imágenes - [NOMBRE_PAÍS]"
# Las imágenes se guardan en la carpeta como "[NÚMERO].jpg" porque los títulos de las obras pueden tener
# caracteres no válidos para un nombre de archivo
# No mostramos la República Checa porque no tiene el mismo esquema en el documento que el resto de países
def buscar_por_pais(tabla):
    pais_escogido = elegir_pais(tabla)
    # Preparar carpeta de imágenes
    nombre_carpeta = str("Imágenes - " + pais_escogido)
    preparar_carpeta_imagenes(nombre_carpeta)
    
    # Mostrar la información de cada uno de las pinturas pertenecientes al país escogido
    indice = 1
    print("\nInformación de las pinturas que se encuentran en " + pais_escogido + ":\n")
    for i in range(len(tabla["titulos"])):
        if pais_escogido == tabla["paises"][i]:
            print(30 * "-" + " " + str(indice) + " " + 30 * "-")
            print("Título:\t" + tabla["titulos"][i])
            print("Fecha:\t" + tabla["fechas"][i])
            print("Dimensiones:\t" + tabla["dimensiones"][i])
            print("Museo:\t" + tabla["museos"][i])
            img = urllib.request.urlopen(tabla["url_imagenes"][i]).read()
            manf = open(nombre_carpeta + "/" + str(indice) + ".jpg", "wb")
            manf.write(img)
            manf.close()
            indice += 1
    print(30 * "-" + " " + "FIN" + " " + 30 * "-")
    print("\nAhora la carpeta '" + nombre_carpeta + "' contiene las imágenes de estas pinturas")

In [ ]:
# Muestra de forma indexada cada uno de los diferentes museos contenidos en la tabla
# Pide al usuario que escoja uno y devuelve su nombre
def elegir_museo(tabla):
    # Mostrar de forma indexada cada uno de los diferentes museos de la tabla
    listaMuseosDiferentes = []
    i = 1
    print("Lista de museos:\n")
    for museo in tabla["museos"]:
        if museo not in listaMuseosDiferentes:
            listaMuseosDiferentes.append(museo)
            print(str(i) + "\t: " + museo)
            i += 1
    # Devolver el museo escogido por el usuario
    print("\nIntroduce un número de museo")
    return listaMuseosDiferentes[int(input()) - 1]

In [ ]:
import urllib.request

# Muestra la informacion de todas las pinturas que se encuentren en el museo que escoja el usuario
# Guarda una imagen de cada pintura en la carpeta "Imágenes - [NOMBRE_MUSEO]"
# Las imágenes se guardan en la carpeta como "[NÚMERO].jpg" porque los títulos de las obras pueden tener
# caracteres no válidos para un nombre de archivo
def buscar_por_museo(tabla):
    museo_escogido = elegir_museo(tabla)
    # Preparar carpeta de imágenes
    nombre_carpeta = str("Imágenes - " + museo_escogido)
    preparar_carpeta_imagenes(nombre_carpeta)
    
    # Mostrar la información de cada uno de las pinturas pertenecientes al museo escogido
    indice = 1
    print("\nInformación de las pinturas que se encuentran en " + museo_escogido + ":\n")
    for i in range(len(tabla["titulos"])):
        if museo_escogido == tabla["museos"][i]:
            print(30 * "-" + " " + str(indice) + " " + 30 * "-")
            print("Título:\t" + tabla["titulos"][i])
            print("Fecha:\t" + tabla["fechas"][i])
            print("Dimensiones:\t" + tabla["dimensiones"][i])
            print("País:\t" + tabla["paises"][i])
            img = urllib.request.urlopen(tabla["url_imagenes"][i]).read()
            manf = open(nombre_carpeta + "/" + str(indice) + ".jpg", "wb")
            manf.write(img)
            manf.close()
            indice += 1
    print(30 * "-" + " " + "FIN" + " " + 30 * "-")
    print("\nAhora la carpeta '" + nombre_carpeta + "' contiene las imágenes de estas pinturas")

In [ ]:
# Muestra el menú de opciones
def print_menu():      
    print(30 * "-" + " MENU " + 30 * "-")
    print("1. Buscar por país")
    print("2. Buscar por museo")
    print("3. Salir")
    print(66 * "-")

In [ ]:
def main():
    tabla = obtener_datos()          
    print_menu()   
    opcion = int(input("Introduce la opción que deseas [1-3]:"))
    if opcion == 1:     
        print("Has seleccionado la opción: Buscar por país\n")
        buscar_por_pais(tabla)
    elif opcion == 2:
        print("Has seleccionado la opción: Buscar por museo\n")
        buscar_por_museo(tabla)
    elif opcion == 3:
        print("Has seleccionado la opción: Salir")
    else:           
        print("Error: Debes seleccionar una opción válida (del 1 al 3)")
main()